In [4]:
import os
import sys
import numpy as np
import nibabel as nib
import matplotlib
import matplotlib.pyplot as plt
import scipy.io

# Ajouter dossier source
sys.path.append("src")
from help_functions_Transform import *

# === PARAMÈTRES ===
case_id = "s0011"
z_min, z_max = 201, 210
seg_dir = f"Data_set/{case_id}/segmentations"
ct_path = f"Data_set/{case_id}/ct.nii.gz"
output_seg_dir = "segmentation_slices"
merged_dir = "merged_masks"
os.makedirs(output_seg_dir, exist_ok=True)
os.makedirs(merged_dir, exist_ok=True)



In [5]:
# === FICHIERS SEGMENTÉS ===
fichiers = [f for f in os.listdir(seg_dir) if f.endswith(".nii.gz")]
ORGANS = { f.rsplit('.nii', 1)[0]: [f] for f in fichiers }

# === DÉTECTION DES ORGANES PRÉSENTS DANS LES SLICES ===
organs_presnt = organs_present_in_crop(seg_dir, z_min, z_max, ORGANS)
print(f"Organes présents entre Z={z_min} et Z={z_max} :")
for org in organs_presnt:
    print(" -", org)


Organes présents entre Z=201 et Z=210 :
 - iliopsoas_left
 - autochthon_right
 - rib_left_10
 - rib_left_12
 - aorta
 - stomach
 - gallbladder
 - duodenum
 - rib_right_12
 - inferior_vena_cava
 - costal_cartilages
 - small_bowel
 - colon
 - vertebrae_L2
 - kidney_left
 - rib_left_11
 - autochthon_left
 - pancreas
 - liver
 - vertebrae_L1
 - iliopsoas_right
 - spinal_cord
 - rib_right_11
 - kidney_right


In [6]:
# === CHARGER LES MASQUES DÉTECTÉS ===
masks = {}
for org in organs_presnt:
    filename = ORGANS[org][0]
    img = nib.load(os.path.join(seg_dir, filename))
    data = img.get_fdata()
    mask_crop = data[:, :, z_min:z_max+1]
    masks[org] = nib.Nifti1Image(mask_crop, affine=img.affine)

# === AJOUT DU SOFT TISSUE ===
mask3d = Get_soft_tissue_mask(case_id, organs_presnt, ORGANS)
soft_tissue = mask3d[0][:, :, z_min:z_max+1]
ref_affine = nib.load(os.path.join(seg_dir, ORGANS[organs_presnt[0]][0])).affine
masks['soft_tissue'] = nib.Nifti1Image(soft_tissue.astype(np.uint8), affine=ref_affine)


Groupe 'ilio' fusionné à partir de 2 organes.
Groupe 'auto' fusionné à partir de 2 organes.
Groupe 'rib_' fusionné à partir de 5 organes.
Groupe 'aort' fusionné à partir de 1 organes.
Groupe 'stom' fusionné à partir de 1 organes.
Groupe 'gall' fusionné à partir de 1 organes.
Groupe 'duod' fusionné à partir de 1 organes.
Groupe 'infe' fusionné à partir de 1 organes.
Groupe 'cost' fusionné à partir de 1 organes.
Groupe 'smal' fusionné à partir de 1 organes.
Groupe 'colo' fusionné à partir de 1 organes.
Groupe 'vert' fusionné à partir de 2 organes.
Groupe 'kidn' fusionné à partir de 2 organes.
Groupe 'panc' fusionné à partir de 1 organes.
Groupe 'live' fusionné à partir de 1 organes.
Groupe 'spin' fusionné à partir de 1 organes.
Groupe 'soft' fusionné à partir de 1 organes.

✅ 17 groupes fusionnés enregistrés dans 'merged_masks'


In [7]:
# === ATTRIBUTION DES COULEURS PAR GROUPE ===
keys = list(merged_masks.keys())
cmap = matplotlib.colormaps.get_cmap('tab20')
group_colors = { key: cmap(i / len(keys)) for i, key in enumerate(keys) }

# === AFFICHAGE ET SAUVEGARDE DES IMAGES COLORÉES PAR SLICE ===
# On récupère la taille exacte du masque pour garder 1 pixel = 1 unité
data0 = merged_masks[keys[0]].get_fdata()
h, w, d = data0.shape

for slice_idx in range(z_min, z_max+1):
    slice_local = slice_idx - z_min
    seg_img = np.zeros((h, w, 4), dtype=float)

    for key, img in merged_masks.items():
        mask_data = img.get_fdata()
        slice_mask = mask_data[:, :, slice_local] > 0
        color = group_colors[key]
        for c in range(4):
            seg_img[..., c][slice_mask] = color[c]

    # FIGURE À LA TAILLE EXACTE (w×h pixels)
    fig, ax = plt.subplots(figsize=(w/100, h/100), dpi=100)
    ax.imshow(seg_img, interpolation='nearest')
    ax.axis('off')

    out_path = os.path.join(output_seg_dir, f"segmented_slice_{slice_idx}.png")
    fig.savefig(out_path, bbox_inches='tight', pad_inches=0)
    plt.close(fig)

print(f"✅ Images segmentées enregistrées dans '{output_seg_dir}' pour les slices {z_min} à {z_max}")


# === SAUVEGARDE DES COULEURS ===
group_names = list(group_colors.keys())
rgb = np.array([group_colors[k][:3] for k in group_names])
scipy.io.savemat('group_colors.mat', {
    'group_names': group_names,
    'rgb_colors': rgb
})
print("✅ Couleurs sauvegardées dans 'group_colors.mat'")


✅ Images segmentées enregistrées dans 'segmentation_slices' pour les slices 201 à 210


In [ ]:
# === SAUVEGARDE DES COULEURS ===
group_names = list(group_colors.keys())
rgb = np.array([group_colors[k][:3] for k in group_names])
scipy.io.savemat('group_colors.mat', {
    'group_names': group_names,
    'rgb_colors': rgb
})
print("✅ Couleurs sauvegardées dans 'group_colors.mat'")

✅ Couleurs sauvegardées dans 'group_colors.mat'


In [ ]:
import os
import sys
import json
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import scipy.io
from skimage import measure
from shapely.geometry import LineString

# --- Paramètres ---
case_id        = "s0011"
z_min, z_max   = 201, 210
seg_dir        = f"Data_set/{case_id}/segmentations"
output_seg_dir = "segmentation_slices"
merged_dir     = "merged_masks"
contour_dir    = "contours"

os.makedirs(output_seg_dir, exist_ok=True)
os.makedirs(merged_dir, exist_ok=True)
os.makedirs(contour_dir, exist_ok=True)

sys.path.append("src")
from help_functions_Transform import *

# 1) Chargement et fusion des masques (votre code existant)
fichiers = [f for f in os.listdir(seg_dir) if f.endswith(".nii.gz")]
ORGANS   = {f.rsplit('.nii',1)[0]: [f] for f in fichiers}

organs_presnt = organs_present_in_crop(seg_dir, z_min, z_max, ORGANS)

# Charger masques et fusion
masks = {}
for org in organs_presnt:
    img  = nib.load(os.path.join(seg_dir, ORGANS[org][0]))
    data = img.get_fdata()[:,:,z_min:z_max+1]
    masks[org] = nib.Nifti1Image((data>0).astype(np.uint8), affine=img.affine)

# Soft tissue
mask3d = Get_soft_tissue_mask(case_id, organs_presnt, ORGANS)[0][:,:,z_min:z_max+1]
ref   = masks[organs_presnt[0]].affine
masks['soft_tissue'] = nib.Nifti1Image(mask3d.astype(np.uint8), affine=ref)

# Grouper puis sauver NIfTI
groups = {}
for org in masks:
    key = 'soft' if org=='soft_tissue' else org[:4]
    groups.setdefault(key, []).append(org)

merged_masks = {}
for key, organs in groups.items():
    arr = np.zeros_like(masks[organs[0]].get_fdata(), dtype=bool)
    for org in organs:
        arr |= (masks[org].get_fdata()>0)
    nii = nib.Nifti1Image(arr.astype(np.uint8), affine=masks[organs[0]].affine)
    path = os.path.join(merged_dir, f"{key}_mask_z{z_min}_{z_max}.nii.gz")
    nib.save(nii, path)
    merged_masks[key] = nii

# 2) Export des contours 2D pour chaque tranche non‐vide
for key, nii in merged_masks.items():
    data = nii.get_fdata()  # shape = (H, W, N_slices)
    for idx in range(data.shape[2]):
        slice_num = z_min + idx
        mask2d = (data[:, :, idx] > 0).astype(np.uint8)
        if mask2d.sum() == 0:
            # masque vide sur cette tranche, on passe
            continue

        # extraction des contours
        contours = measure.find_contours(mask2d, 0.5)
        if not contours:
            print(f"[Contours] {key} slice {slice_num}: aucun contour trouvé, skipped")
            continue

        # on prend le contour le plus long
        contour = max(contours, key=lambda c: c.shape[0])
        poly = [list(pt[::-1]) for pt in contour]

        # simplification
        ls  = LineString(poly)
        ls2 = ls.simplify(2.0, preserve_topology=False)
        poly2 = list(ls2.coords)

        # fermeture de la boucle si nécessaire
        if np.linalg.norm(np.array(poly2[0]) - np.array(poly2[-1])) > 1e-3:
            poly2.append(poly2[0])

        # sauvegarde .mat, avec numéro de tranche
        outname = os.path.join(contour_dir, f"{key}_contour_z{slice_num}.mat")
        scipy.io.savemat(outname, {"poly": np.array(poly2)})
        #print(f"[Contours] {key} slice {slice_num}: {len(poly2)} points")


# 3) Optionnel: générer les images segmentées pour visuel
keys = list(merged_masks.keys())
cmap = plt.get_cmap('tab20')
colors = {k: cmap(i/len(keys)) for i,k in enumerate(keys)}

h,w,_ = merged_masks[keys[0]].shape
for idx,sl in enumerate(range(z_min,z_max+1)):
    seg_rgba = np.zeros((h,w,4))
    for key in keys:
        mask2d = merged_masks[key].get_fdata()[:,:,idx]>0
        seg_rgba[mask2d] = colors[key]
    fig,ax = plt.subplots(figsize=(w/100,h/100),dpi=100)
    ax.imshow(seg_rgba,interpolation='nearest'); ax.axis('off')
    plt.savefig(os.path.join(output_seg_dir,f"seg_{sl}.png"),
                bbox_inches='tight',pad_inches=0)
    plt.close(fig)


In [ ]:
import numpy as np

def detailed_overlaps(merged_masks, z_min):
    """
    Parcourt toutes les tranches de merged_masks entre z_min et z_max, collecte les chevauchements
    et affiche :
      - le total pairwise (somme des pixels pour chaque paire d’organes),
      - le total unique  (nombre de voxels distincts recoupés par ≥2 masques).

    Retourne :
      - overlaps : liste de dicts détaillant chaque chevauchement pair-à-pair, avec :
          • slice_z      : numéro réel de la tranche
          • organ_pair   : (org1, org2)
          • area_pixels  : nombre de pixels communs pour cette paire
          • bbox_pixels  : (xmin, ymin, xmax, ymax) de la zone d’intersection
          • coords_pixels: liste des [row, col] si area_pixels < 50, sinon None
    """
    keys = list(merged_masks.keys())
    overlaps = []
    n_slices = merged_masks[keys[0]].shape[2]

    # --- Collecte des chevauchements pairwise ---
    for idx in range(n_slices):
        z = z_min + idx
        # extraire chaque masque 2D booléen
        masks2d = {
            k: (merged_masks[k].get_fdata()[:, :, idx] > 0)
            for k in keys
        }
        # tester toutes les paires
        for i in range(len(keys)):
            for j in range(i+1, len(keys)):
                k1, k2 = keys[i], keys[j]
                inter = np.logical_and(masks2d[k1], masks2d[k2])
                if not inter.any():
                    continue

                coords = np.argwhere(inter)  # [[row, col], ...]
                area = int(inter.sum())
                ymin, xmin = coords.min(axis=0)
                ymax, xmax = coords.max(axis=0)

                overlaps.append({
                    'slice_z'      : z,
                    'organ_pair'   : (k1, k2),
                    'area_pixels'  : area,
                    'bbox_pixels'  : (int(xmin), int(ymin), int(xmax), int(ymax)),
                    'coords_pixels': coords.tolist() if area < 50 else None
                })

    # --- Calcul et affichage des totaux ---
    total_pairwise = sum(item['area_pixels'] for item in overlaps)
    print(f"\nTotal pairwise (toutes paires) : {total_pairwise}")

    # compter les voxels uniques recoupés par ≥2 masques
    total_unique = 0
    for idx in range(n_slices):
        # empiler tous les masques sur cette tranche
        stack = np.stack([
            merged_masks[k].get_fdata()[:, :, idx] > 0
            for k in keys
        ], axis=0)  # shape = (n_masks, H, W)
        cover_count = stack.sum(axis=0)    # nombre de masques couvrant chaque pixel
        total_unique += np.sum(cover_count >= 2)

    print(f"Total unique   (voxels distincts) : {total_unique}\n")

    return overlaps


# ─── Exemple d’utilisation ─────────────────────────────────────────────────

# Supposons que merged_masks et z_min sont définis dans votre scope :
info = detailed_overlaps(merged_masks, z_min)

# Pour afficher le détail :
for item in info:
    z      = item['slice_z']
    a, b   = item['organ_pair']
    area   = item['area_pixels']
    bbox   = item['bbox_pixels']
    print(f"Tranche z={z} : {a} ∩ {b} → {area} px, bbox={bbox}")
    if item['coords_pixels'] is not None:
        print("  coords:", item['coords_pixels'])


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# extrait les pixels où au moins 3 masques se recoupent
cover3 = np.zeros_like(merged_masks[keys[0]].get_fdata()[:,:,0], dtype=bool)
for idx in range(10):
    stack = np.stack([merged_masks[k].get_fdata()[:,:,idx]>0 for k in keys],axis=0)
    cover3_slice = stack.sum(axis=0) >= 3
    if cover3_slice.any():
        cover3 = cover3_slice
        print(f"Overlap ≥3 sur tranche z={z_min+idx}")
        plt.imshow(cover3, cmap='gray')
        plt.title(f"Recouvrement ≥3 masques à z={z_min+idx}")
        plt.axis('off')
        plt.show()
        break
